# Spoken commands example
This example uses an audio classifier model from a Tensorflow tutorial:
https://www.tensorflow.org/tutorials/sequences/audio_recognition

In [ ]:
import sys
sys.path.append('..')
from pathlib import Path
import tarfile
import numpy as np
from scipy.io.wavfile import read, write
from src.problemgenerator.series import Series
from src.problemgenerator.tuple import Tuple
from src.problemgenerator.filters import ClipWAV, ApplyToTuple

In [ ]:
data_url = "https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz"
fname = "speech_commands_v0.02.tar.gz"
data_dir = Path.home() / "datasets/speech_data"

In [ ]:
if not data_dir.exists():
    !mkdir -p {data_dir}
    !wget {data_url} -P {data_dir}
    tarfile.open(data_dir / fname, "r:gz").extractall(data_dir)

In [ ]:
example_path = Path.home() / "dpEmu/src/examples/speech_commands"
data_subset_dir = data_dir / "off"

In [ ]:
fs = list(data_subset_dir.iterdir())
wavs = [read(f) for f in data_subset_dir.iterdir()]

In [ ]:
wav_node = Tuple()
wav_node.addfilter(ApplyToTuple(ClipWAV("dyn_range"), 1))
root_node = Series(wav_node)
clipped = root_node.generate_error(wavs, {"dyn_range": .3})

In [ ]:
example_index = 22  # Arbitrarily chosen speech command example – try changing the index!

In [ ]:
write('clipped.wav', 16000, clipped[example_index][1])

In [ ]:
!aplay {fs[example_index]}

In [ ]:
!aplay clipped.wav

In [ ]:
scores_clean = !python {example_path}/label_wav.py \
--graph={example_path}/trained_model/my_frozen_graph.pb \
--labels={example_path}/trained_model/conv_labels.txt \
--wav={fs[example_index]}

In [ ]:
scores_clipped = !python {example_path}/label_wav.py \
--graph={example_path}/trained_model/my_frozen_graph.pb \
--labels={example_path}/trained_model/conv_labels.txt \
--wav='clipped.wav'

In [ ]:
# Define a function to filter out irrelevant output (e.g. Python deprecation warnings)
def filter_scores(output):
    return [line for line in output if "score" in line or ".wav" in line]

In [ ]:
filter_scores(scores_clean)

In [ ]:
filter_scores(scores_clipped)

In [ ]:
# You can also obtain the scores for an entire directory of .wav files in one command
scores_clean_dir = !python {example_path}/label_wav_dir.py \
--graph={example_path}/trained_model/my_frozen_graph.pb \
--labels={example_path}/trained_model/conv_labels.txt \
--wav_dir={data_subset_dir}

filter_scores(scores_clean_dir)